In [1]:
from functools import wraps

In [3]:
def logger(fn):
    @wraps(fn)
    def wrapped(*args, **kwargs):
        print(f'Log: {fn.__name__} called')
        return fn(*args, **kwargs)
    return wrapped

In [4]:
@logger
def say_hello():
    pass

In [5]:
say_hello()

Log: say_hello called


In [11]:
class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called')
        return self.fn(*args, **kwargs)

In [16]:
def say_hello():
    return 'hello'

In [17]:
f = Logger(say_hello)

In [18]:
f

In [19]:
f()

Log: say_hello called


'hello'

In [20]:
@Logger
def say_hello():
    return 'hello'

In [21]:
say_hello()

Log: say_hello called


'hello'

In [23]:
class Person():
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f'{self.name} says hello'
    

In [24]:
p = Person('lol')

In [25]:
p.say_hello()

Log: say_hello called


TypeError: say_hello() missing 1 required positional argument: 'self'

In [26]:
p.say_hello

In [28]:
Person.__init__

<function __main__.Person.__init__(self, name)>

In [29]:
hasattr(Person.__init__, '__get__')

True

In [30]:
hasattr(Person.say_hello, '__get__')

False

In [31]:
from types import MethodType

In [34]:
class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called')
        return self.fn(*args, **kwargs)
    
    
    def __get__(self, instance, owner_class):
        print(f'__get__ called: self = {self}, instance = {instance}')
        if instance is None:
            print('\treturning self unbound...')
            return self
        else:
            print('\treturning self as a method bound to instance')
            return MethodType(self, instance) #here self is inst of logger and it has call method
        
        

In [35]:
class Person:
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f'{self.name} says hello'

In [36]:
p = Person('lol')

In [37]:
p.say_hello

__get__ called: self = <__main__.Logger object at 0x0000015F26AF7A30>, instance = <__main__.Person object at 0x0000015F27073610>
	returning self as a method bound to instance


<bound method ? of <__main__.Person object at 0x0000015F27073610>>

In [38]:
p.say_hello()

__get__ called: self = <__main__.Logger object at 0x0000015F26AF7A30>, instance = <__main__.Person object at 0x0000015F27073610>
	returning self as a method bound to instance
Log: say_hello called


'lol says hello'

In [39]:
@Logger
def say_bye():
    pass

In [40]:
say_bye

In [41]:
say_bye()

Log: say_bye called


In [44]:
class Person:
    @classmethod
    @Logger
    def class_method(cls):
        print('class method called')
        
    @staticmethod
    @Logger
    def static_method():
        print('static method called')

In [46]:
Person.class_method()

__get__ called: self = <__main__.Logger object at 0x0000015F2718D7F0>, instance = <class '__main__.Person'>
	returning self as a method bound to instance
Log: class_method called
class method called


In [47]:
Person.static_method()

Log: static_method called
static method called


In [49]:
#### MEtaclasses VS CLass Decorators

In [50]:
def fn_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner


def class_logger(cls):
    for name , obj in vars(cls).items():
        if callable(obj):
            print('decorating', cls, name)
            setattr(cls, name, fn_logger(obj))
    return cls
    




In [51]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f'hello, my name is {self.name} and i am {self.age} old'
    
        

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [52]:
Person('lol', 10).greet()

Log: Person.__init__((<__main__.Person object at 0x0000015F2725B400>, 'lol', 10), {}) = None
Log: Person.greet((<__main__.Person object at 0x0000015F2725B400>,), {}) = hello, my name is lol and i am 10 old


'hello, my name is lol and i am 10 old'

In [53]:
class ClassLogger(type):
    def __new__(mcls, name, bases, class_dict):
        cls = super().__new__(mcls, name, bases, class_dict)
        for name , obj in cls.__dict__.items():
            if callable(obj):
                print('decorating', cls, name)
                setattr(cls, name, fn_logger(obj))
        return cls
        

In [56]:
class Person(metaclass = ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f'hello, my name is {self.name} and i am {self.age} old'
    
        
    

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [57]:
p = Person('john', 78).greet()

Log: Person.__init__((<__main__.Person object at 0x0000015F276D4E80>, 'john', 78), {}) = None
Log: Person.greet((<__main__.Person object at 0x0000015F276D4E80>,), {}) = hello, my name is john and i am 78 old


In [58]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f'hello, my name is {self.name} and i am {self.age} old'
    
    

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [59]:
class Student(Person):
    def __init__(self, name, age, st_no):
        super().__init__(name, age)
        self.st_no = st_no
        
    def study(self):
        return f'{self.name} studies'

In [60]:
s = Student('lol', 19, 'abjb')

Log: Person.__init__((<__main__.Student object at 0x0000015F274CA610>, 'lol', 19), {}) = None


In [61]:
s.study()

'lol studies'

In [62]:
@class_logger
class Student(Person):
    def __init__(self, name, age, st_no):
        super().__init__(name, age)
        self.st_no = st_no
        
    def study(self):
        return f'{self.name} studies'

decorating <class '__main__.Student'> __init__
decorating <class '__main__.Student'> study


In [63]:
s = Student('lol', 19, 'abjb')

Log: Person.__init__((<__main__.Student object at 0x0000015F277890D0>, 'lol', 19), {}) = None
Log: Student.__init__((<__main__.Student object at 0x0000015F277890D0>, 'lol', 19, 'abjb'), {}) = None


In [64]:
s.study()

Log: Student.study((<__main__.Student object at 0x0000015F277890D0>,), {}) = lol studies


'lol studies'

In [68]:
class Person(metaclass = ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f'hello, my name is {self.name} and i am {self.age} old'
    

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [69]:
class Student(Person):
    def __init__(self, name, age, st_no):
        super().__init__(name, age)
        self.st_no = st_no
        
    def study(self):
        return f'{self.name} studies'
    

decorating <class '__main__.Student'> __init__
decorating <class '__main__.Student'> study


In [70]:
s = Student('lpl', 90, 'lol')

Log: Person.__init__((<__main__.Student object at 0x0000015F272CB9A0>, 'lpl', 90), {}) = None
Log: Student.__init__((<__main__.Student object at 0x0000015F272CB9A0>, 'lpl', 90, 'lol'), {}) = None


In [71]:
s.study()

Log: Student.study((<__main__.Student object at 0x0000015F272CB9A0>,), {}) = lpl studies


'lpl studies'

In [72]:
class Metaclass1(type):
    pass

In [73]:
class Metaclalss2(type):
    pass

In [74]:
class Person(metaclass = Metaclass1):
    pass

In [75]:
class Student(Person, metaclass = Metaclalss2):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [76]:
class Person(metaclass = Metaclass1):
    pass

In [77]:
class AccHolder(metaclass = Metaclalss2):
    pass

In [78]:
class BankAccPerson(Person, AccHolder):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [79]:
###MEtaclass Usage

In [80]:
##META PARAM

In [81]:
class Metaclass(type):
    def __new__(mcls, name, bases, class_dict):
        return super().__new__(mcls, name, bases, class_dict)
    
class MyClass(metaclass = Metaclass):
    pass

In [83]:
type(MyClass), type(MyClass())

(__main__.Metaclass, __main__.MyClass)

In [84]:
class Metaclass(type):
    def __new__(mcls, name, bases, class_dict, arg1, arg2, arg3 = None):
        print(arg1, arg2, arg3)
        return super().__new__(mcls, name, bases, class_dict)
    

In [85]:
class MyClass(object, metaclass= Metaclass, arg1 = 10,arg2 = 20,arg3=30):
    pass

10 20 30


In [93]:
class AutoClassAttr(type):
    def __new__(mcls, name, bases, class_dict, extra_attrs=None):
        if extra_attrs:
            print('Creating class with some extra attr:', extra_attrs)
            for attr_name, attr_val in extra_attrs:
                class_dict[attr_name] = attr_val
        return super().__new__(mcls, name, bases, class_dict)
                

In [94]:
class Acc(metaclass=AutoClassAttr, extra_attrs=(('acc_type', 'savings'), ('apr', 0.5))):
    pass

Creating class with some extra attr: (('acc_type', 'savings'), ('apr', 0.5))


In [95]:
vars(Acc)

mappingproxy({'__module__': '__main__',
              'acc_type': 'savings',
              'apr': 0.5,
              '__dict__': <attribute '__dict__' of 'Acc' objects>,
              '__weakref__': <attribute '__weakref__' of 'Acc' objects>,
              '__doc__': None})

In [96]:
class AutoClassAttr(type):
    def __new__(mcls, name, bases, class_dict, extra_attrs=None):
        print('Creating class with some extra attr:', extra_attrs)
        new_cls = super().__new__(mcls, name, bases, class_dict)
        if extra_attrs:
            for attr_name, attr_val in extra_attrs:
                setattr(new_cls,attr_name, attr_val)
        return new_cls
                

In [97]:
class Acc(metaclass=AutoClassAttr, extra_attrs=(('acc_type', 'savings'), ('apr', 0.5))):
    pass

Creating class with some extra attr: (('acc_type', 'savings'), ('apr', 0.5))


In [98]:
Acc.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Acc' objects>,
              '__weakref__': <attribute '__weakref__' of 'Acc' objects>,
              '__doc__': None,
              'acc_type': 'savings',
              'apr': 0.5})

In [110]:
class AutoClassAttr(type):
    def __new__(mcls, name, bases, class_dict, **extra_attrs):
        print('Creating class with some extra attr:', extra_attrs)
        new_cls = super().__new__(mcls, name, bases, class_dict)
        if extra_attrs:
            for attr_name, attr_val in extra_attrs.items():
                setattr(new_cls,attr_name, attr_val)
        return new_cls
                

In [111]:
class Acc(metaclass= AutoClassAttr, acc_type = 'Savings', apr = 0.5):
    pass

Creating class with some extra attr: {'acc_type': 'Savings', 'apr': 0.5}


In [112]:
vars(Acc)

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Acc' objects>,
              '__weakref__': <attribute '__weakref__' of 'Acc' objects>,
              '__doc__': None,
              'acc_type': 'Savings',
              'apr': 0.5})

In [113]:
class AutoClassAttr(type):
    def __new__(mcls, name, bases, class_dict, **extra_attrs):
        print('Creating class with some extra attr:', extra_attrs)
        class_dict.update(extra_attrs)
        new_cls = super().__new__(mcls, name, bases, class_dict)
        
        return new_cls
                

In [114]:
class Acc(metaclass= AutoClassAttr, acc_type = 'Savings', apr = 0.5):
    pass

Creating class with some extra attr: {'acc_type': 'Savings', 'apr': 0.5}


In [115]:
vars(Acc)

mappingproxy({'__module__': '__main__',
              'acc_type': 'Savings',
              'apr': 0.5,
              '__dict__': <attribute '__dict__' of 'Acc' objects>,
              '__weakref__': <attribute '__weakref__' of 'Acc' objects>,
              '__doc__': None})

In [116]:
###  __prepare__ method

In [120]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('MyMeta.__new__ called')
        print('\tcls:', mcls, type(mcls))
        print('\tname:', name, type(name))
        print('\tbases:', bases, type(bases))
        print('\tcls_dict:' ,cls_dict, type(cls_dict))
        print('\tkwargs:' , kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [121]:
class MyClass(metaclass = MyMeta):
    pass

MyMeta.__new__ called
	cls: <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases: () <class 'tuple'>
	cls_dict: {'__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {}


In [124]:
class MyMeta(type):
    @staticmethod
    def __prepare__(name, bases, **kwargs):
        print('MyMeta.__prepare called')
        print('\tname:' ,name)
        print('\tkwargs', kwargs)
        return {'a':100, 'b':200}
    
    
    @staticmethod
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('MyMeta.__new__ called')
        print('\tcls:', mcls, type(mcls))
        print('\tname:', name, type(name))
        print('\tbases:', bases, type(bases))
        print('\tcls_dict:' ,cls_dict, type(cls_dict))
        print('\tkwargs:' , kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [126]:
class MyClass(metaclass = MyMeta, kw1=1, kw2=2):
    pass

MyMeta.__prepare called
	name: MyClass
	kwargs {'kw1': 1, 'kw2': 2}
MyMeta.__new__ called
	cls: <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases: () <class 'tuple'>
	cls_dict: {'a': 100, 'b': 200, '__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {'kw1': 1, 'kw2': 2}


In [128]:
type(type.__prepare__())

dict

In [129]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        cls_dict.update(kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [130]:
class MyClass(metaclass = MyMeta, kw1=100, kw2=200):
    pass

In [131]:
vars(MyClass)

mappingproxy({'__module__': '__main__',
              'kw1': 100,
              'kw2': 200,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [139]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs):
        return {'a':100, **kwargs}
    
    def __new__(mcls,name,bases, cls_dict, **kwargs):
        return super().__new__(mcls,name, bases, cls_dict)

In [140]:
class MyClass(metaclass = MyMeta, arg1 = 1, arg2 = 2):
    pass

In [141]:
vars(MyClass)

mappingproxy({'a': 100,
              'arg1': 1,
              'arg2': 2,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [146]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs):
        return 100
    
    def __new__(mcls,name,bases, cls_dict, **kwargs):
        return super().__new__(mcls,name, bases, cls_dict)

In [147]:
class MyClass(metaclass = MyMeta, arg1 = 1, arg2 = 2):
    pass

ERROR! Session/line number was not unique in database. History logging moved to new session 184


TypeError: MyMeta.__prepare__() must return a mapping, not int

In [148]:
from collections import OrderedDict


In [151]:
class MyMeta(type):
    def __prepare__(name, bases):
        d = OrderedDict()
        d['bonus']='rocky'
        return d

In [150]:
isinstance(OrderedDict(), dict)

True

In [152]:
class MyClass(metaclass = MyMeta):
    pass

In [153]:
vars(MyClass)

mappingproxy({'bonus': 'rocky',
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [154]:
from collections import UserDict

In [155]:
class CustomDict(UserDict):
    def __setitem__(self, k,v):
        print(f'setting {k}={v} in cus dict')
        super().__setitem__(k,v)
        
    def __getitem__(self, k):
        print(f'Getting {k} from cus dict')
        return int(super().__getitem__(k))

In [156]:
class MyMeta(type):
    def __prepare__(name, bases):
        return CustomDict()

In [157]:
class MyClass(metaclass = MyMeta):
    pass

Getting __name__ from cus dict
setting __module__=__main__ in cus dict
setting __qualname__=MyClass in cus dict


TypeError: type.__new__() argument 3 must be dict, not CustomDict

In [158]:
issubclass(UserDict, dict)

False

In [159]:
class CustomDict(dict):
    def __setitem__(self, k,v):
        print(f'setting {k}={v} in cus dict')
        super().__setitem__(k,v)
        
    def __getitem__(self, k):
        print(f'Getting {k} from cus dict')
        return int(super().__getitem__(k))

In [160]:
class MyMeta(type):
    def __prepare__(name, bases):
        return CustomDict()

In [161]:
class MyClass(metaclass = MyMeta):
    pass

Getting __name__ from cus dict
setting __module__=__main__ in cus dict
setting __qualname__=MyClass in cus dict


In [ ]:
##CLASSES , METACLSSE